In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
idg = ImageDataGenerator()

In [4]:
train_generator = idg.flow_from_directory('data/', 
                                          target_size = (256, 256), 
                                          batch_size = 16, 
                                          class_mode = None,
                                          color_mode = 'grayscale')

Found 37692 images belonging to 2 classes.


In [5]:
type(train_generator)

keras.preprocessing.image.DirectoryIterator

In [14]:
noise_dim = 100
channels = 1
optimizer = Adam(0.0002, 0.5)
img_rows, img_cols = 256, 256

In [6]:
batch = train_generator.next()

In [7]:
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten
from keras.layers import LeakyReLU, BatchNormalization, Dropout, Input, RandomFlip
from keras.initializers import RandomNormal
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.utils import image_dataset_from_directory
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [8]:
def create_generator():
    generator = Sequential()
    
    # Starting size
    d = 4
    generator.add(Dense(d*d*256, kernel_initializer = RandomNormal(0, 0.02), input_dim = noise_dim))
    generator.add(LeakyReLU(0.2))
    generator.add(Reshape((d, d, 256)))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2DTranspose(128, (4, 4), strides = 2, padding = 'same', 
                                  kernel_initializer = RandomNormal(0, 0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Conv2D(channels, (2, 2), padding = 'same', activation = 'tanh', 
                         kernel_initializer = RandomNormal(0, 0.02)))
    
    generator.compile(loss = binary_crossentropy, optimizer = optimizer)
    return generator

In [9]:
def create_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Input(shape=(img_cols, img_rows, channels)))
    discriminator.add(RandomFlip('horizontal_and_vertical'))
    discriminator.add(Conv2D(64, (3, 3), padding = 'same', kernel_initializer = RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(128, (3, 3), strides = 2, padding = 'same', kernel_initializer = RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(128, (3, 3), strides = 2, padding = 'same', kernel_initializer = RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(128, (3, 3), strides = 2, padding = 'same', kernel_initializer = RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Conv2D(256, (3, 3), strides = 2, padding = 'same', kernel_initializer = RandomNormal(0, 0.02)))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Flatten())
    discriminator.add(Dropout(0.4))
    discriminator.add(Dense(1, activation = 'sigmoid', input_shape = (img_cols, img_rows, channels)))
    
    discriminator.compile(loss = binary_crossentropy, optimizer = optimizer)
    return discriminator

In [10]:
def create_gan():
    generator = create_generator()
    discriminator = create_discriminator()

    discriminator.trainable = False

    gan_input = Input(shape=(noise_dim,))
    fake_image = generator(gan_input)

    gan_output = discriminator(fake_image)

    gan = Model(gan_input, gan_output)
    gan.compile(loss = binary_crossentropy, optimizer = optimizer)
    
    return generator, discriminator, gan

In [18]:
def train_gan(generator, discriminator, gan, image_generator, epochs, batch_size):
    steps_per_epoch = math.floor(image_generator.__len__() * batch_size)
    
    discriminator_labels = np.zeros(2 * batch_size)
    discriminator_labels[:batch_size] = 0.9
    generator_labels = np.ones(batch_size)
    
    for epoch in range(epochs):
        for batch in range(steps_per_epoch):
            noise = np.random.normal(0, 1, size = (batch_size, noise_dim))
            
            fake_images = generator.predict(noise)
            real_images = image_generator.next()
            combined_images = np.concatenate((real_images, fake_images))

            discriminator_loss = discriminator.train_on_batch(combined_images, discriminator_labels)
            generator_loss = gan.train_on_batch(noise, generator_labels)

        print(f'Epoch: {epoch} \t Discriminator Loss: {discriminator_loss} \t\t Generator Loss: {generator_loss}')
        
    return generator, discriminator, gan

In [19]:
generator, discriminator, gan = train_gan(*create_gan(), 
                                          image_generator = train_generator, 
                                          epochs = 1, 
                                          batch_size = 16)

1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step


KeyboardInterrupt: 